In [1]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from openmmtools import cache, utils
from perses.dispersed.utils import configure_platform
cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import argparse
import copy
from perses.dispersed import feptasks
import numpy as np
from simtk.openmm import app
from openmmforcefields.generators import SystemGenerator
import pickle
import mdtraj as md
import itertools

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA


test the energy discrepancy of the alanine dipeptide in vacuum

In [2]:
from openmmtools.testsystems import AlanineDipeptideVacuum
ala = AlanineDipeptideVacuum()

In [3]:
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


In [3]:
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]

In [5]:
rest_atoms

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [6]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b779898bb10> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b779898b8a0> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b779898b270> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b779898b6f0> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x2b779898bdb0> >]

In [4]:
ala.system.removeForce(4) #remove CMM

In [8]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2b779898b930> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2b779898bb70> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2b77ce392810> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b77ce392360> >]

In [7]:
class REST2(RESTTopologyFactory):
    """
    subclass REST, but move solvent-solvent to custom nonbonded force
    """

    def _add_nonbonded_force_terms_v2(self):
        standard_nonbonded_force = openmm.NonbondedForce()
        self._out_system.addForce(standard_nonbonded_force)
        self._out_system_forces[standard_nonbonded_force.__class__.__name__] = standard_nonbonded_force

        #set the appropriate parameters
        epsilon_solvent = self._og_system_forces['NonbondedForce'].getReactionFieldDielectric()
        r_cutoff = self._og_system_forces['NonbondedForce'].getCutoffDistance()
        if self._nonbonded_method != openmm.NonbondedForce.NoCutoff:
            standard_nonbonded_force.setReactionFieldDielectric(epsilon_solvent)
            standard_nonbonded_force.setCutoffDistance(r_cutoff)
        if self._nonbonded_method in [openmm.NonbondedForce.PME, openmm.NonbondedForce.Ewald]:
            [alpha_ewald, nx, ny, nz] = self._og_system_forces['NonbondedForce'].getPMEParameters()
            delta = self._og_system_forces['NonbondedForce'].getEwaldErrorTolerance()
            standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
            standard_nonbonded_force.setEwaldErrorTolerance(delta)
        standard_nonbonded_force.setNonbondedMethod(self._nonbonded_method)

        if self._og_system_forces['NonbondedForce'].getUseDispersionCorrection() and self._use_dispersion_correction:
            self._out_system_forces['NonbondedForce'].setUseDispersionCorrection(True)
        else:
            self._out_system_forces['NonbondedForce'].setUseDispersionCorrection(False)
        
        if self._og_system_forces['NonbondedForce'].getUseSwitchingFunction():
            switching_distance = self._og_system_forces['NonbondedForce'].getSwitchingDistance()
            standard_nonbonded_force.setUseSwitchingFunction(True)
            standard_nonbonded_force.setSwitchingDistance(switching_distance)
        else:
            standard_nonbonded_force.setUseSwitchingFunction(False)


        #add the global value
        self._out_system_forces['NonbondedForce'].addGlobalParameter('solute_scale', 0.)
        self._out_system_forces['NonbondedForce'].addGlobalParameter('inter_scale', 0.)

    def _add_nonbondeds_v2(self):
        self._solute_exceptions, self._interexceptions = [], []
        og_nb_force = self._og_system_forces['NonbondedForce']

        for particle_idx in range(self._num_particles):
            q, sigma, epsilon = og_nb_force.getParticleParameters(particle_idx)
            identifier = self.get_identifier(particle_idx)

            if identifier == 1: #solvent
                self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)

            else: #solute
                self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)
                self._out_system_forces['NonbondedForce'].addParticleParameterOffset('electrostatic_scale', particle_idx, q, 0.0*sigma, epsilon*0.0)
                self._out_system_forces['NonbondedForce'].addParticleParameterOffset('steric_scale', particle_idx, q*0.0, 0.0*sigma, epsilon)


        #handle exceptions
        for exception_idx in range(og_nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = og_nb_force.getExceptionParameters(exception_idx)
            identifier = self.get_identifier([p1, p2])
#             self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
            if identifier == 1: #solvent
                exc_idx = self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
                #self._out_system_forces['NonbondedForce'].addExceptionParameterOffset('steric_scale', exc_idx, chargeProd, 0.0*sigma, epsilon)
            elif identifier == 0: #solute
                self._solute_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                exc_idx = self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
                self._out_system_forces['NonbondedForce'].addExceptionParameterOffset('steric_scale', exc_idx, chargeProd, 0.0*sigma, epsilon)
            elif identifier == 2: #inter
                self._interexceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                exc_idx = self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['NonbondedForce'].addExceptionParameterOffset('electrostatic_scale', exc_idx, chargeProd, 0.0*sigma, epsilon)
                #self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions


In [8]:
rtf = REST2(system = ala.system, solute_region=rest_atoms, use_dispersion_correction=False)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [9]:
# ss = SamplerState(positions=ala.positions)

In [9]:
T_min = 298*unit.kelvin

In [10]:
# og_thermostate = ThermodynamicState(ala.system, temperature=300*unit.kelvin)
# og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
# og_context = og_thermostate.create_context(og_integrator)
# ss.apply_to_context(og_context)
# ss.update_from_context(og_context)
# print(ss.potential_energy)

og_thermostate = ThermodynamicState(ala.system, temperature=T_min)
og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
og_context = og_thermostate.create_context(og_integrator)
og_context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(og_context)
og_thermostate.reduced_potential(sampler_state)

-35.552365146022886

In [11]:
thermostate = ThermodynamicState(rtf.REST_system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
thermostate.reduced_potential(sampler_state)
# ss.apply_to_context(context)
# ss.update_from_context(context)

-57.888273322178264

In [13]:
from perses.tests.utils import compute_potential_components

In [14]:
og_forces = [q[1] for q in compute_potential_components(og_context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [15]:
new_forces = [q[1] for q in compute_potential_components(context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [16]:
og_forces[0] - new_forces[0] #bond

0.0

In [17]:
og_forces[1] - new_forces[1] #angle

0.0

In [18]:
og_forces[2] - new_forces[2] #torsion

-4.349742189990735e-06

In [19]:
np.array(og_forces[3:]).sum() - np.array(new_forces[3:]).sum() #nonbonded

-1.6881940467783352e-06

test the energy discrepancy of the alanine dipeptide in solvent

In [42]:
from openmmtools.testsystems import AlanineDipeptideExplicit
ala = AlanineDipeptideExplicit()

In [43]:
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]

In [44]:
rest_atoms

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [45]:
ala.system.getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2af23a7f7f90> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2af23a7f7d20> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2af23a7f7e10> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2af23a7f7d50> >,
 <simtk.openmm.openmm.CMMotionRemover; proxy of <Swig Object of type 'OpenMM::CMMotionRemover *' at 0x2af23a7f7c30> >]

In [46]:
ala.system.removeForce(4) #remove CMM

In [47]:
ala.system.getForce(3).getNonbondedMethod()

4

In [50]:
ala.system.getForce(3).getUseSwitchingFunction()

True

In [49]:
ala.system.getForce(3).getCutoffDistance() 

Quantity(value=1.0, unit=nanometer)

In [51]:
ala.system.getForce(3).getSwitchingDistance()

Quantity(value=0.8500000000000001, unit=nanometer)

In [52]:
ala.system.getForce(3).getUseDispersionCorrection() 

True

In [53]:
ala.system.getForce(3).getExceptionsUsePeriodicBoundaryConditions()

AttributeError: type object 'object' has no attribute '__getattr__'

In [48]:
openmm.NonbondedForce.PME

4

In [25]:
ala.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)

In [26]:
rtf = RESTTopologyFactory(system = ala.system, solute_region=rest_atoms, use_dispersion_correction=False)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


In [27]:
# ss = SamplerState(positions=ala.positions)

In [28]:
T_min = 298*unit.kelvin

In [29]:
# og_thermostate = ThermodynamicState(ala.system, temperature=300*unit.kelvin)
# og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
# og_context = og_thermostate.create_context(og_integrator)
# ss.apply_to_context(og_context)
# ss.update_from_context(og_context)
# print(ss.potential_energy)
og_thermostate = ThermodynamicState(ala.system, temperature=T_min)
og_integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
og_context = og_thermostate.create_context(og_integrator)
og_context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(og_context)
og_thermostate.reduced_potential(sampler_state)

-9896.170979940161

In [30]:
thermostate = ThermodynamicState(rtf.REST_system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
# ss.apply_to_context(context)
# ss.update_from_context(context)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
thermostate.reduced_potential(sampler_state)

-9896.17097563446

In [31]:
og_forces = [q[1] for q in compute_potential_components(og_context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [32]:
new_forces = [q[1] for q in compute_potential_components(context, thermostate.beta)]

conducting subsequent work with the following platform: CUDA


In [33]:
og_forces[0] - new_forces[0] #bond

0.0

In [34]:
og_forces[1] - new_forces[1] #angle

0.0

In [35]:
og_forces[2] - new_forces[2] #torsion

-3.3203714742136015e-06

In [36]:
np.array(og_forces[3:]).sum() - np.array(new_forces[3:]).sum() #nonbonded

-9.853301889961585e-07

In [37]:
og_forces

[0.03478366095963102,
 0.6112088544716918,
 3.2515208598833776,
 -9900.068493315477,
 0.0]

In [38]:
new_forces

[0.03478366095963102,
 0.6112088544716918,
 3.251524180254852,
 -9859.230542518875,
 -148.62845838958208,
 107.79050857831042,
 0.0]

the discrepancy is clearly coming from the nonbonded force; are these exceptions?

Create a subclass of the RESTTopologyFactory so that we can play around with the functions and test the energies

In [5]:
class RESTTEST(RESTTopologyFactory):
    """
    subclass REST, but manually set and scale per particle parameters with the given naught and scaled betas
    """

    def __init__(self, system, solute_region, use_dispersion_correction=False, beta0, beta_m):
        self._beta0 = beta0
        self._beta_m = beta_m
        super().__init__(system, solute_region, use_dispersion_correction)\
    
    def bonded_scale(self, identifier):
        if identifier==0:
            out = self._beta_m/self._beta0
        elif identifier==1:
            out = 1.
        elif identifier==2:
            out= np.sqrt(self._beta_m/self._beta0)
        else:
            raise Exception as e:
                print(e)
        return out
        
    
    def _add_bond_force_terms(self):
        force = openmm.HarmonicBondForce()
        self._out_system.addForce(force)
        self._out_system_forces[force.__class__.__name__] = force
    
    def _add_angle_force_terms(self):
        force = openmm.HarmonicAngleForce()
        self._out_system.addForce(force)
        self._out_system_forces[force.__class__.__name__] = force
    
    def _add_torsion_force_terms(self):
        force = openmm.PeriodicTorsionForce()
        self._out_system.addForce(force)
        self._out_system_forces[force.__class__.__name__] = force
    
    def _add_nonbonded_force_terms(self)
        
        epsilon_solvent = self._og_system_forces['NonbondedForce'].getReactionFieldDielectric()
        r_cutoff = self._og_system_forces['NonbondedForce'].getCutoffDistance()
        
        if self._nonbonded_method != openmm.NonbondedForce.NoCutoff:
            standard_nonbonded_force.setReactionFieldDielectric(epsilon_solvent)
            standard_nonbonded_force.setCutoffDistance(r_cutoff)
        if self._nonbonded_method in [openmm.NonbondedForce.PME, openmm.NonbondedForce.Ewald]:
            [alpha_ewald, nx, ny, nz] = self._og_system_forces['NonbondedForce'].getPMEParameters()
            delta = self._og_system_forces['NonbondedForce'].getEwaldErrorTolerance()
            standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
            standard_nonbonded_force.setEwaldErrorTolerance(delta)
        standard_nonbonded_force.setNonbondedMethod(self._nonbonded_method)
        
        if self._og_system_forces['NonbondedForce'].getUseSwitchingFunction():
            switching_distance = self._og_system_forces['NonbondedForce'].getSwitchingDistance()
            standard_nonbonded_force.setUseSwitchingFunction(True)
            standard_nonbonded_force.setSwitchingDistance(switching_distance)
        else:
            standard_nonbonded_force.setUseSwitchingFunction(False)
    
    def _add_bonds(self):
        force = self._og_bond_forces['HarmonicBondForce']
        for bond_idx in range(force.getNumBonds()):
            p1, p2, length, k = force.getBondParameters(bond_idx)
            identifier = self.get_identifier([p1, p2])
            scale = self.bonded_scale(identifier)
            self._out_system_forces['HarmonicBondForce'].addBond(p1, p2, length, k*scale)
    
    def _add_angles(self):
        force = self._og_bond_forces['HarmonicAngleForce']
        for angle_idx in range(force.getNumAngles()):
            p1, p2, p3, theta0, k = force.getAngleParameters(angle_idx)
            identifier = self.get_identifier([p1, p2, p3])
            scale = self.bonded_scale(identifier)
            self._out_system_forces['HarmonicAngleForce'].addAngle(p1, p2, p3, theta0, k*scale)
    
    def _add_torsions(self):
        force = self._og_bond_forces['PeriodicTorsionForce']
        for torsion_idx in range(force.getNumTorsions(force)):
            p1, p2, p3, p4, per, phase, k = force.getTorsionParameters(torsion_idx)
            identifier = self.get_identifier([p1, p2, p3, p4])
            scale = self.bonded_scale(identifier)
            self._out_system_forces['PeriodicTorsionForce'].addTorsion(p1, p2, p3, p4, per, phase, k*scale)
    
    def _add_nonbondeds(self):
        force = self._og_system_forces
        for particle_idx in range
    
            
            
        
        
        
        
    
        
    def _add_nonbonded_force_terms(self):
        from openmmtools.constants import ONE_4PI_EPS0 # OpenMM constant for Coulomb interactions (implicitly in md_unit_system units)
        standard_nonbonded_force = openmm.NonbondedForce()
        self._out_system.addForce(standard_nonbonded_force)
        self._out_system_forces[standard_nonbonded_force.__class__.__name__] = standard_nonbonded_force
        
        
    def _add_nonbondeds(self):
        self._solute_exceptions, self._interexceptions = [], []

        #the output nonbonded force _only_ contains solvent atoms (the rest are zeroed); same with exceptions
        """
        First, handle the NonbondedForce in the out_system
        """
        og_nb_force = self._og_system_forces['NonbondedForce']
        for particle_idx in range(self._num_particles):
            q, sigma, epsilon = og_nb_force.getParticleParameters(particle_idx)
            identifier = self.get_identifier(particle_idx)

            if identifier == 1:
                self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])
            else:
                self._out_system_forces['NonbondedForce'].addParticle(q*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])

        #add appropriate interaction group
        solute_ig, solvent_ig = set(self._solute_region), set(self._solvent_region)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solvent_ig)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solute_ig)

        #handle exceptions
        for exception_idx in range(og_nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = og_nb_force.getExceptionParameters(exception_idx)
            identifier = self.get_identifier([p1, p2])
            if identifier == 1:
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 0:
                self._solute_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 2:
                self._interexceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions

        #now add the CustomBondForce for exceptions
        exception_force = self._out_system_forces['CustomExceptionForce']

        for solute_exception_term in self._solute_exceptions:
            p1, p2, [chargeProd, sigma, epsilon] = solute_exception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 0
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

        for interexception_term in self._interexceptions:
            p1, p2, [chargeProd, sigma, epsilon] = interexception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 2
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

In [6]:
# Create REST system
factory = REST2(sys, solute_region=list(range(0, 22)))
REST_system = factory.REST_system


getting into the nb version


Get the energy of the REST system at the thermodynamic state where energy is scaled to 600K

In [39]:
T_min = 298.0 *unit.kelvin
T = 600.0 * unit.kelvin

# Create thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(rtf.REST_system)
thermostate = ThermodynamicState(rtf.REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
compound_thermodynamic_state.reduced_potential(sampler_state)


-9893.550764403697

Get energy of vanilla system (after manually scaling energies) at same thermostate as above

In [40]:
system = ala.system

# Compute energy for non-RESTified system
protein = rest_atoms
environment = [i for i in range(ala.topology.getNumAtoms()) if i not in rest_atoms]
protein_scaling = beta_m / beta_0
inter_scaling = np.sqrt(beta_m / beta_0)

# Scale the terms in the bond force appropriately
bond_force = system.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in protein and p2 in protein:
        bond_force.setBondParameters(bond, p1, p2, length, k * protein_scaling)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

# Scale the terms in the angle force appropriately
angle_force = system.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in protein and p2 in protein and p3 in protein:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * protein_scaling)
    elif set([p1, p2, p3]).intersection(set(protein)) != set() and set([p1, p2, p3]).intersection(set(environment)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

# Scale the terms in the torsion force appropriately
torsion_force = system.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in protein and p2 in protein and p3 in protein and p4 in protein:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * protein_scaling)
    elif set([p1, p2, p3, p4]).intersection(set(protein)) != set() and set([p1, p2, p3, p4]).intersection(set(environment)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# Scale the exceptions in the nonbonded force appropriately
nb_force = system.getForce(3)
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in protein and p2 in protein:
        nb_force.setExceptionParameters(nb_index, p1, p2, protein_scaling * chargeProd, sigma, protein_scaling * epsilon)
    elif (p1 in protein and p2 in environment) or (p1 in environment and p2 in protein):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)

# Scale nonbonded interactions for solute region by adding exceptions for all pairs of atoms 
exception_pairs = [tuple(sorted([nb_force.getExceptionParameters(nb_index)[0], nb_force.getExceptionParameters(nb_index)[1]])) for nb_index in range(nb_force.getNumExceptions())]
solute_pairs = set([tuple(sorted(pair)) for pair in list(itertools.product(protein, protein))])
for pair in list(solute_pairs):
    p1 = pair[0]
    p2 = pair[1]
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if p1 != p2:
        if pair not in exception_pairs:
#             print(pair)
            nb_force.addException(p1, p2, p1_charge*p2_charge*protein_scaling, 0.5*(p1_sigma+p2_sigma), np.sqrt(p1_epsilon*p2_epsilon)*protein_scaling)
        
# Scale nonbonded interactions for inter region by adding exceptions for all pairs of atoms 
# traj = md.Trajectory(np.array(positions), md.Topology.from_openmm(solvated_topology))
# nearby_waters = md.compute_neighbors(traj, 0.4, list(range(0,22)), haystack_indices=list(range(22, solvated_topology.getNumAtoms())))[0]
for pair in list(itertools.product(protein, environment)):
    p1 = pair[0]
    p2 = int(pair[1]) # otherwise, will be a numpy int
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if tuple(sorted(pair)) not in exception_pairs:
        nb_force.addException(p1, p2, p1_charge*p2_charge*inter_scaling, 0.5*(p1_sigma+p2_sigma), np.sqrt(p1_epsilon*p2_epsilon)*inter_scaling)

# Save energy
thermostate = ThermodynamicState(system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
context = thermostate.create_context(integrator)
context.setPositions(ala.positions)
sampler_state = SamplerState.from_context(context)
thermostate.reduced_potential(sampler_state)

-9893.55076491989